In [ ]:
import time
import paho.mqtt.client
import paho.mqtt.publish
import numpy as np
import mkl_random as random
import datetime
import json
import ssl
import sys
import psycopg2
from sqlalchemy import create_engine

In [ ]:
host='lallah.db.elephantsql.com'
user ='yirsxctz'
password='AS3sYxp2fpRwAczZa1OZpIzI4etWHM6n'
dbname='yirsxctz'

In [ ]:
myConnection = psycopg2.connect(host = host,
                                user= user, password =password,
                                dbname= dbname)

In [ ]:
def on_connect(client, userdata, flags, rc):    
    print('connected (%s)' % client._client_id)
    client.subscribe(topic='Tienda/#', qos = 2)  
def on_message(client, userdata, message):   
    a = json.loads(message.payload)
    print(a)
    print('------------------------------') 
def select(query):
    cur = myConnection.cursor()
    try:
        cur.execute(query)
    except Exception as e:
        myConnection.commit()
        print('Error en el query:', e)
    else:
        records = cur.fetchall()
        cur.close()
        return records
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return random.randint(range_start, range_end)

In [ ]:
host = "broker.hivemq.com"

client = paho.mqtt.client.Client("Equipo", False)

client.qos = 2

client.message_callback_add('Tienda/EntradaClientes', on_message)

client.connect(host=host)

client.on_connect = on_connect

client.loop_start()

In [ ]:
query_clientes = '''select * from cliente_afiliado'''
numero_clientes = len(select(query_clientes))
nombres = ['Fernando', 'Juan', 'Raul', 'Manuel', 'Samuel', 'Gustavo', 'Eduardo', 'Carlos', 'Manuel', 'Arnoldo', 'María', 'José', 'Marcos', 'Bárbara', 'Tatiana', 'Sara', 'Antonia', 'Mauricio', 'Lara', 'Mérida']
apellidos = ['Acosta', 'Acuña', 'Aguilar', 'Aguirre', 'Agustín', 'Ahumada', 'Alanis', 'Alarcón', 'Alayón', 'Alcázar', 'Alcocer', 'Alfaro', 'Almendárez', 'Altamirano', 'Álvarez', 'Alzate', 'Amador', 'Anaya', 'Argüelles', 'Arjona', 'Arriaga', 'Arrollo', 'Ayala']

while (True):
    #random_time = int(random.randint(6))
    numero_compras = int(random.randint(1,6))
    tienda_escogida = int(random.randint(1,3))
    estante = 0
    productos_comprados = []
    cantidades = []
    productosxestante = []
    cambios_precio_productos = []
    afiliado = int(random.randint(2))
    productos = []
    estantes = []
    sePuedeComprar = []
    
    for compra in range(1, 3):
        
        query_estante = '''select distinct(id_estante) from productoxestante'''
        for x in range(0, len(select(query_estante))):
            estante = select(query_estante)[x][0]
            estantes.append(estante)
        
        if (tienda_escogida == 1):
            estante = int(random.randint(1,6))
        else:
            estante = int(random.randint(6,11))
        
        estante = 2
        query_productos = '''select id_producto from productoxestante where id_estante='''+str(estante)+''''''
        for x in range(0, len(select(query_productos))):
            producto = select(query_productos)[x][0]
            productos.append(producto)
        producto_escogido = int(random.randint(0, len(productos)))
        productos_comprados.append(productos[producto_escogido])
        
        query_productosxestante = '''select id_productoxestante from productoxestante where id_producto='''+str(productos[producto_escogido])+''';'''
        productoxestante = select(query_productosxestante)[0][0]
        productosxestante.append(productoxestante)
        
        query_cantidad = '''select * from producto_disponiblexestante
                            where id_productoxestante = '''+str(productoxestante)+'''
                            order by id_producto_disponiblexestante desc
                            limit 1'''
        cantidad_disponible = select(query_cantidad)[0][0]
        
        if (cantidad_disponible == 0):
            sePuedeComprar.append(0)
        else:
            sePuedeComprar.append(1)
        
        cantidad_seleccionada = int(random.randint(1, cantidad_disponible + 1))
        cantidades.append(cantidad_seleccionada)
        
        query_precio = '''select id_cambio_precio_producto from cambio_precio_producto
                            where id_producto = '''+str(productos[producto_escogido])+'''
                            order by fecha desc
                            limit 1'''
        precio = select(query_precio)[0][0]
        cambios_precio_productos.append(precio)
    
    #datos cliente nuevo
    nombre_random=int(random.randint(0, len(nombres)))
    apellido_random=int(random.randint(0, len(apellidos)))
    
    date_base = datetime.datetime.now()
    payload = {
        #visitas conocidas
        "afiliado": afiliado,
        "cliente": int(random.randint(1, numero_clientes)),
        "tienda": tienda_escogida,
        "fecha": str(date_base),
        
        #nombre cliente nuevo
        "nombre_cliente": nombres[nombre_random],
        "apellido_cliente": apellidos[apellido_random],
        "direccion_cliente": "Caracas",
        "cedula_cliente": random_with_N_digits(8),
        "telefono_cliente": random_with_N_digits(8),
        
        #compras
        #"id_factura": 1,
        "productos": productos_comprados,
        "cantidades": cantidades,
        "cambios_precio_productos": cambios_precio_productos,
        "productosxestante": productosxestante,
        "sePuedeComprar": sePuedeComprar
    }

    client.publish('Tienda/EntradaClientes',json.dumps(payload),qos=0)
    time.sleep(3)